In [1]:
import os
import pandas as pd

from pathlib import Path
from itertools import product

# lv share charging demand

In [2]:
ele_dir = Path(
    r"\\192.168.10.221\Daten_flexibel_02\simbev_results\eDisGo_object_files"
)

scenarios = [
    "NEP_C_2035",
    "Reference_2050",
    "Electrification_2050_sensitivity_low_work",
    "Electrification_2050",
]

grid_ids = ["176", "177", "1056", "1690", "1811", "2534"]

sub_dir = "dumb"

sub_sub_dir = "topology"
sub_sub_dir_ts = "timeseries"

file_cps = "charging_points.csv"
file_ts = "charging_points_active_power.csv"

data_dirs = [
    Path(os.path.join(ele_dir, scenario, grid_id, sub_dir, sub_sub_dir, file_cps))
    for scenario in scenarios for grid_id in grid_ids
]

In [3]:
def cartesian_product(arr1, arr2):  
    # return the list of all the computed tuple 
    # using the product() method 
    return list(product(arr1, arr2))

In [4]:
data = cartesian_product(scenarios, grid_ids)
df_connection = pd.DataFrame(data=data, columns=["scenario", "grid_id"])
df_connection["lv_share_p"] = 0
df_connection["lv_share_e"] = 0

In [5]:
try:
    df_connection = pd.read_csv("lv_share_grid_connection.csv", index_col=[0])
    
except:
    for data_dir in data_dirs:
        scenario = data_dir.parts[-5]
        grid_id = data_dir.parts[-4]

        df_cps = pd.read_csv(data_dir, index_col=[0])

        ts_dir = Path(
            os.path.join(
                data_dir.parent.parent,
                sub_sub_dir_ts,
                file_ts
            )        
        )

        df_ts = pd.read_csv(ts_dir, index_col=[0])

        cols = df_cps[df_cps.p_nom <= 0.3].index.tolist()

        df_ts_lv = df_ts[cols]

        lv_share_p = df_cps[df_cps.p_nom <= 0.3].p_nom.sum() / df_cps.p_nom.sum()
        lv_share_e = df_ts_lv.sum().sum() / df_ts.sum().sum()

        df_connection.loc[
            (df_connection.scenario == scenario) &
            (df_connection.grid_id == grid_id)
        ] = df_connection.loc[
            (df_connection.scenario == scenario) &
            (df_connection.grid_id == grid_id)
        ].assign(
            lv_share_p=lv_share_p,
            lv_share_e=lv_share_e,
        )
        
    df_connection.to_csv("lv_share_grid_connection.csv")
    

# lv grid with largest charging share

In [31]:
data = cartesian_product(scenarios, grid_ids)
df_largest_lv = pd.DataFrame(data=data, columns=["scenario", "grid_id"])
df_largest_lv["lv_grids_count"] = 0
df_largest_lv["share"] = 0

file_transformers = r"transformers.csv"

In [32]:
try:
    df_largest_lv = pd.read_csv("cp_share_of_largest_lv_grid.csv", index_col=[0])
    
except:
    for data_dir in data_dirs:
        scenario = data_dir.parts[-5]
        grid_id = data_dir.parts[-4]

        ts_dir = Path(
            os.path.join(
                data_dir.parent.parent,
                sub_sub_dir_ts,
                file_ts
            )        
        )

        df_ts = pd.read_csv(ts_dir, index_col=[0])

        transformer_dir = Path(
            os.path.join(
                data_dir.parent,
                file_transformers,
            )        
        )

        df_transformer = pd.read_csv(transformer_dir, index_col=[0])

        transformer_indexes = list(set([transformer.split("_")[-2] for transformer in df_transformer.index.tolist()]))

        df_ts_T = df_ts.T

        share_list = []

        for idx in transformer_indexes:
            share_list.append(df_ts_T[df_ts_T.index.str.contains(idx)].sum().sum())

        max_e_lv = max(share_list)

        total_e_lvs = sum(share_list)

        df_largest_lv.loc[
                (df_largest_lv.scenario == scenario) &
                (df_largest_lv.grid_id == grid_id)
            ] = df_largest_lv.loc[
                (df_largest_lv.scenario == scenario) &
                (df_largest_lv.grid_id == grid_id)
            ].assign(
                lv_grids_count=len(transformer_indexes),
                share=max_e_lv/total_e_lvs,
            )

    df_largest_lv.to_csv("cp_share_of_largest_lv_grid.csv")


In [33]:
df_largest_lv

,scenario,grid_id,lv_grids_count,share
0,NEP_C_2035,176,105,0.353403
1,NEP_C_2035,177,56,0.259750
2,NEP_C_2035,1056,130,0.129397
3,NEP_C_2035,1690,179,0.111494
4,NEP_C_2035,1811,381,0.075140
5,NEP_C_2035,2534,9,0.623027
6,Reference_2050,176,105,0.362688
7,Reference_2050,177,56,0.265474
8,Reference_2050,1056,130,0.127388
9,Reference_2050,1690,179,0.111005
